# Introdução

A análise de dados financeiros desempenha um papel crucial na tomada de decisões nos mercados. Neste contexto, a engenharia de atributos surge como uma abordagem fundamental para extrair informações relevantes e aprimorar o desempenho dos modelos de machine learning aplicados a séries temporais financeiras. Este projeto aborda a engenharia de atributos em dados de mercado, explorando técnicas avançadas para melhorar a representação dos dados e, consequentemente, a qualidade das previsões.

In [2]:
#Bibliotecas
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.metrics import precision_score

import plotly.graph_objects as go
import plotly.express as px

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Importando dados preprocessados

In [3]:
df = pd.read_csv("data/BTC-USD.csv")
df.set_index("Date", inplace=True)
df.head(5)

,Open,High,Low,Close,Volume,Returns,Range,Equity Curve,Bench_C_Rets,RSI,...,MA_21,DOW,Roll_Rets,Avg_Range,Returns_T1,Range_T1,RSI_Ret_T1,Returns_T2,Range_T2,RSI_Ret_T2
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-31,0.000878,0.053054,0.001615,970.403015,1.103895,0.054348,0.055442,-0.027969,-0.050254,60.205366,...,885.613522,1,0.011408,0.057910,0.000964,0.003887,1.003974,-0.002272,0.004646,0.991144
2017-02-01,0.054272,0.017587,0.054056,989.023010,-0.087932,0.019188,0.018926,-0.009318,-0.032030,63.109498,...,895.673808,2,0.007132,0.057381,0.054348,0.055442,1.192634,0.000964,0.003887,1.003974
2017-02-02,0.019630,0.024675,0.012855,1011.799988,-0.028572,0.023030,0.030816,0.013498,-0.009738,66.345006,...,905.529332,3,0.008542,0.057675,0.019188,0.018926,1.048237,0.054348,0.055442,1.192634
2017-02-03,0.021676,0.020079,0.026005,1029.910034,0.380309,0.017899,0.024861,0.031638,0.007986,68.695922,...,915.335333,4,-0.079792,0.054832,0.023030,0.030816,1.051268,0.019188,0.018926,1.048237
2017-02-04,0.019645,0.011636,0.006314,1042.900024,-0.229603,0.012613,0.030281,0.044650,0.020700,70.298572,...,926.025239,5,0.055230,0.045565,0.017899,0.024861,1.035435,0.023030,0.030816,1.051268


# Adicionando Variável Target

In [4]:
# Especificando Target
df.loc[df["Range"].shift(-1) > df["Avg_Range"], "TARGET"] = 1 # Quando Range seguinte for maior que média o modelo classifica como 1
df.loc[df["Range"].shift(-1) <= df["Avg_Range"], "TARGET"] = 0 # Quando Range for igual ou menor que a média o modelo classifica como 0

In [5]:
# Verificando Valores Nulos (NaN)
nan_location = np.where(np.isnan(df))
print(nan_location)

(array([2476], dtype=int64), array([22], dtype=int64))


In [6]:
# Preenchendo NaN
df["TARGET"].fillna(0, inplace=True)
df.tail()

,Open,High,Low,Close,Volume,Returns,Range,Equity Curve,Bench_C_Rets,RSI,...,DOW,Roll_Rets,Avg_Range,Returns_T1,Range_T1,RSI_Ret_T1,Returns_T2,Range_T2,RSI_Ret_T2,TARGET
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-08,0.010605,0.002842,0.017426,35655.277344,-0.081729,0.005973,0.024087,34.715100,33.896283,79.056775,...,2,0.264974,0.032208,0.011593,0.038980,1.023620,-0.000342,0.014977,0.997738,1.0
2023-11-09,0.006046,0.053671,0.012641,36693.125000,1.183395,0.029108,0.065581,35.754689,34.912038,82.787702,...,3,0.301067,0.033888,0.005973,0.024087,1.011473,0.011593,0.038980,1.023620,0.0
2023-11-10,0.029989,-0.011403,0.021652,37313.968750,-0.398579,0.016920,0.031105,36.376574,35.519666,84.559697,...,4,0.336887,0.033779,0.029108,0.065581,1.047193,0.005973,0.024087,1.011473,0.0
2023-11-11,0.016561,-0.002313,0.011300,37138.050781,-0.386900,-0.004715,0.017225,36.200361,35.347493,81.984170,...,5,0.336508,0.033898,0.016920,0.031105,1.021404,0.029108,0.065581,1.047193,0.0
2023-11-12,-0.004719,-0.004796,0.000148,37054.519531,-0.170821,-0.002249,0.012196,36.116690,35.265740,80.726854,...,6,0.330313,0.033797,-0.004715,0.017225,0.969542,0.016920,0.031105,1.021404,0.0


# Dividindo em Treino e Teste

In [7]:
# Removendo colunas indesejadas
df_tts = df.copy()
df_tts.drop(columns=["Close", "Bench_C_Rets", "Open", "High", "Low"], inplace=True)

In [8]:
# Divindo em Learning (X) and Target (y) os dados
X = df_tts.iloc[:, : -1]
y = df_tts.iloc[:, -1]

In [9]:
# Performando Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)

Shape of X_train:  (1981, 17)
Shape of y_train:  (1981,)


# Construindo Modelo inicial

In [10]:
# Selecionando modelo para otimização
is_binary = True
is_optimise_for_precision = True

In [11]:
# Determinando Objetivo e Métricas de Avaliação
if is_binary:
    objective = "binary:logistic"
    eval_metric = "logloss"
    eval_metric_list = ["error", "logloss", eval_metric]
else:
    objective = "multi:softmax"
    eval_metric = "mlogloss"
    eval_metric_list = ["merror", "mlogloss", eval_metric]

In [12]:
# Refinando as Métricas de Avaliação
if is_binary and is_optimise_for_precision:
    eval_metric = "aucpr"
    scoring = "precision"
elif is_binary and not is_optimise_for_precision:
    eval_metric = "auc"
    scoring = "f1"
else:
    scoring = "accuracy"

In [13]:
# Construindo o Primeiroo Modelo de Classificação - Model 0
classifier_0 = XGBClassifier(
    objective=objective,
    booster="gbtree",
    eval_metric=eval_metric,
    subsample=0.8,
    colsample_bytree=1,
    random_state=1,
    use_label_encoder=False
)

# Ajustando Hyperparameters

In [14]:
# Grid de Hyperparams
param_grid = {
    "gamma": [0, 0.1, 0.2, 0.5, 1, 1.5, 2, 3, 6, 12, 20],
    "learning_rate": [0.01, 0.02, 0.03, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8],
    "max_depth": [1, 2, 3, 4, 5, 6, 8, 12],
    "n_estimators": [25, 50, 65, 80, 100, 115, 200]
}

In [15]:
# Usando técnica 'Random Search' para encontrar os melhores Hyperparameters

grid_search = RandomizedSearchCV(estimator=classifier_0, param_distributions=param_grid, scoring=scoring)
best_model = grid_search.fit(X_train, y_train)
hyperparams = best_model.best_params_
ne = hyperparams["n_estimators"]
lr = hyperparams["learning_rate"]
md = hyperparams["max_depth"]
gm = hyperparams["gamma"]
print("Hiperparâmetros Recomendados >>", f"ne: {ne},", f"lr: {lr}", f"md: {md}", f"gm: {gm}")

Hiperparâmetros Recomendados >> ne: 80, lr: 0.03 md: 2 gm: 6


# Treinando o Modelo

In [16]:
# Construindo Modelo de Classificação - Model 1
classifier_1 = XGBClassifier(
    objective=objective,
    booster="gbtree",
    eval_metric=eval_metric,
    n_estimators=ne,
    learning_rate=lr,
    max_depth=md,
    gamma=gm,
    subsample=0.8,
    colsample_bytree=1,
    random_state=1,
    use_label_encoder=False
)

In [17]:
# Treinando o Modelo
eval_set = [(X_train, y_train)]
classifier_1.fit(
    X_train,
    y_train,
    eval_set=eval_set,
    verbose=False
)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric='aucpr', feature_types=None, gamma=6,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=80, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

# Avaliação de Performance (Análise rápida)

In [18]:
# Previsões para dados de treino
train_yhat = classifier_1.predict(X_train)
print("Training Preds: \n", train_yhat[:5])

Training Preds: 
 [1 0 0 0 1]


In [19]:
# Implementando K-Fold Cross Validation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

In [20]:
# Treinando Resultados
train_results = cross_val_score(classifier_1, X_train, y_train, scoring=scoring, cv=cv, n_jobs=1)

In [21]:
# Breve Revisão dos Resultados do Treinamento
print("Acurácia Média K-Fold: ", round(train_results.mean(), 2))
print("Desvio Padrão K-Fold: ", round(train_results.std(), 2))
print("Pontuação de Precisão 0 (igual ou abaixo da média do 'Range'): ", round(precision_score(y_train, train_yhat, average=None)[0], 3))
print("Pontuação de Precisão 1 (acima da média do 'Range'): ", round(precision_score(y_train, train_yhat, average=None)[1], 3))
print("")
print("Apenas para referência. No momento, estamos focados apenas em obter algumas características iniciais.")

Acurácia Média K-Fold:  0.7
Desvio Padrão K-Fold:  0.06
Pontuação de Precisão 0 (igual ou abaixo da média do 'Range'):  0.693
Pontuação de Precisão 1 (acima da média do 'Range'):  0.788

Apenas para referência. No momento, estamos focados apenas em obter algumas características iniciais.


# Seleção de Atributos

In [22]:
# Dados de importância
importance_labels = X.columns
importance_features = classifier_1.feature_importances_

# Criar DataFrame
importance_df = pd.DataFrame({'Atributos': importance_labels, 'Importância': importance_features})

# Ordenar por importância
importance_df = importance_df.sort_values(by='Importância', ascending=False)

# Criar figura com Plotly Express
fig = px.bar(importance_df, x='Atributos', y='Importância', color='Importância',
             labels={'Importância': 'Grau de Importância'},
             color_continuous_scale='Jet', height=500, width=1000)

# Atualizar layout
fig.update_layout(
    title='Importância dos Atributos',
    xaxis=dict(title='Atributos'),
    yaxis=dict(title='Importância'),
    coloraxis_colorbar=dict(title='Grau de Importância'),
)

# Exibir gráfico interativo
fig.show()

# Desenvolvimento: Engenharia de Atributos

A análise das características e suas importâncias é uma parte crucial ao realizar engenharia de atributos para modelos de machine learning. Vamos explorar o significado dessas características e o impacto delas na importância, conforme fornecido nos resultados:

RSI (Relative Strength Index): O Índice de Força Relativa é uma medida de momentum que avalia a velocidade e a magnitude de mudanças recentes nos preços. Uma alta importância sugere que variações nas condições de sobrecompra ou sobrevenda podem influenciar significativamente o modelo.

RSI_Ret (RSI Return): Esta característica está relacionada à variação percentual do RSI em relação a um período anterior. Pode indicar a taxa de mudança do RSI, sendo útil para capturar tendências emergentes.

Returns (Retornos): Representa a variação percentual no preço do ativo em um determinado período. Esta é uma medida fundamental e frequentemente uma das características mais importantes, pois reflete diretamente o desempenho do ativo.

Range: O alcance é a diferença entre o preço mais alto e o mais baixo durante um período. Uma alta importância sugere que a amplitude das flutuações de preços é um fator significativo para o modelo.

DOW (Dia da Semana): Se os dados incluem o dia da semana, a importância associada pode indicar padrões específicos nos diferentes dias. Pode ser usado para capturar comportamentos ou volatilidades específicas aos dias úteis.

Volume: O volume de negociação é a quantidade total de ações ou contratos negociados em um título durante um período específico. A importância associada a isso indica que as mudanças no volume podem influenciar nas decisões de negociação.

Essas características, com suas respectivas importâncias, fornecem insights sobre como o modelo percebe e utiliza diferentes aspectos dos dados para tomar decisões. A interpretação e a seleção dessas características desempenham um papel crucial na eficácia do modelo preditivo.

In [23]:
# Seleção dos Melhores Atributos
mean_feature_importance = importance_features.mean()
i = 0
recommended_feature_labels = []
recommended_feature_score = []
for fi in importance_features:
    if fi > mean_feature_importance:
        recommended_feature_labels.append(importance_labels[i])
        recommended_feature_score.append(fi)
    i += 1

In [24]:
# Criar DataFrame com os atributos recomendados
recommended_feature_df = pd.DataFrame({
    'Características': recommended_feature_labels,
    'Importância': recommended_feature_score
})

# Ordenar o DataFrame pela importância dos atributos
recommended_feature_df = recommended_feature_df.sort_values(by='Importância', ascending=False)

# Criar o gráfico de barras interativo
fig = px.bar(recommended_feature_df, x='Características', y='Importância',
             labels={'Importância': 'Grau de Importância'},
             title='Características Recomendadas',
             color='Importância', color_continuous_scale='Jet',
             height=500, width=1000)

# Adicionar título e rótulos dos eixos
fig.update_layout(title_text='Características Recomendadas',
                  xaxis_title='Características',
                  yaxis_title='Grau de Importância')

# Exibir o gráfico interativo
fig.show()

# Considerações Finais

A implementação de técnicas avançadas de engenharia de atributos e a seleção criteriosa de modelos são passos cruciais na análise de dados financeiros. Este projeto oferece uma visão abrangente de como essas técnicas podem ser aplicadas para aprimorar a representação dos dados temporais e melhorar a eficácia dos modelos preditivos. Ao continuar refinando essas abordagens, é possível aprimorar ainda mais a capacidade de previsão e, assim, tomar decisões mais informadas no cenário financeiro dinâmico.